## Q1. Running Elastic

In [83]:
!curl localhost:9200

{
  "name" : "9cfe4484ed5c",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "g_BzYMVsTb6yRulPsztm1g",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Value for ```version.build_hash```: 42f05b9372a9a4a470db3b52817899b99a76ee73

In [84]:
# from the given snippet
import requests 
from tqdm import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in tqdm(documents_raw):
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 6849.71it/s]


## Q2. Indexing the data

In [85]:
from elasticsearch import Elasticsearch

In [86]:
es_client = Elasticsearch("http://localhost:9200")

In [87]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [88]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework"

In [90]:
# es_client.indices.delete(index=index_name)

In [91]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework'})

To add new documents to the elastic index we use the ```index``` function.

In [92]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:10<00:00, 90.55it/s]


In [105]:
def elastic_search(query, 
                   size: int = 4, 
                   boost: int = 4, 
                   filters: dict | None = None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": [f"question^{boost}", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": filters
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return response

## Q3. Searching

In [101]:
query = "How do execute a command on a Kubernetes pod?"

In [106]:
result_docs = elastic_search(query)

In [107]:
result_docs = sorted(result_docs['hits']['hits'], key=lambda d: d['_score'], reverse=True)

In [108]:
top_result_score = [d['_score'] for d in result_docs]
top_result_score

[44.50556, 35.433445, 33.70974, 33.2635]

The score for the top ranking result is **44.50**

## Q4. Filtering

In [141]:
query = "How do copy a file to a Docker container?"

In [142]:
filters = {"term": {"course": "machine-learning-zoomcamp"}}

result_docs = elastic_search(query, size=3, filters=filters)

In [143]:
result_docs['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

The 3rd returned question is: *How do I copy files from a different folder into docker container’s working directory?*

## Q5. Building a prompt

In [144]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [145]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [146]:
query = 'How do I execute a command in a running docker container?'

In [147]:
context = ""

for doc in result_docs['hits']['hits']:
    doc = doc['_source']
    context += context_template.format(question=doc['question'], text=doc['text'])

prompt = prompt_template.format(question=query, context=context).strip()
len(prompt)

1458

In [149]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/p

## Q6. Tokens

In [150]:
import tiktoken

In [151]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [153]:
len(encoding.encode(prompt))

321